<a href="https://colab.research.google.com/github/Berenice2018/DeepLearning/blob/master/PySyft_OxfordFlower102_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tried to train on the Oxford flower 102 dataset. But, it takes too long to distribute the images to PySyft workers in CoLab. 

There also seems to be an issue with BatchNorm layers in PySyft. So, pre-trained models are not yet usable. Thus, a deep NN with Convolutional Layers does not work yet smoothly. 

Trying with a simpler dataset, in another notebook. 

### Imports, setup


In [0]:
#!pip install syft

#!rm -rf './PySyft-Bc'

In [0]:
import time
import datetime
import logging
import math
import os
import sys

# save the model on Google Drive, link Google drive to this notebook
from google.colab import drive
drive.mount('/content/gdrive')


# After executing this cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/gdrive/My Drive/Colab Notebooks/flower_data/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
picco_test  test  train  train_ofgdrive  valid	valid_ofgdrive


In [0]:
!pip install tf-encrypted
! URL="https://github.com/Berenice2018/PySyft-Bc.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b master --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft-Bc; python setup.py install

module_path = os.path.abspath(os.path.join('./PySyft-Bc'))
if module_path not in sys.path:
     sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

In [0]:

import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset

import syft as sy


logger = logging.getLogger(__name__)

print(torch.cuda.is_available())

W0724 13:26:34.586156 140499686274944 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0724 13:26:34.607257 140499686274944 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



True


In [0]:
# check the manually edited file to fix a Pysyft bug
#!cat './PySyft-Bc/syft/frameworks/torch/pointers/pointer_tensor.py'

In [0]:
# paths to training and test data
data_dir = '/content/gdrive/My Drive/Colab Notebooks/flower_data/'

# use a smaller set of images, since the GPU will not be used. 
# https://github.com/OpenMined/PySyft/issues/1893

train_dir = data_dir + 'valid'
valid_dir = data_dir + 'test'

#os.chdir("/content/gdrive/My Drive/Colab Notebooks/")
#test_dir = data_dir + 'test'

### Architecture and helpers

In [0]:
# Make data loader based on the selected pre-trained model
def create_loaders(base, final = False):
    print('returning datasets')
    # ResNet, DenseNet expect 224, Inception expects 299
    img_size = 299 if base == 'Inception' else 224 


    transforms_train = transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    transforms_test = transforms.Compose([
        transforms.Resize(img_size + 1),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    # Load the datasets with ImageFolder
    trainset = datasets.ImageFolder(train_dir, transform=transforms_train)
    validationset = datasets.ImageFolder(valid_dir, transform=transforms_train)
    testset = datasets.ImageFolder(valid_dir, transform=transforms_test)
       
    return trainset, validationset, testset

In [0]:
#transforms a torch.Dataset or a sy.BaseDataset into a sy.FederatedDataset. 
def dataset_federate(dataset, workers):
    print('dataset_federate')

    datasets = []
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=32)
    
    for dataset_idx, (data, targets) in enumerate(data_loader):
        worker = workers[dataset_idx % len(workers)]
        data = data.send(worker)
        targets = targets.send(worker)
        datasets.append(sy.BaseDataset(data, targets))
    
    fed_dataset = sy.FederatedDataset(datasets)
    fed_loader = sy.FederatedDataLoader(fed_dataset, batch_size=32, shuffle=False, drop_last=False)
    
    return fed_loader

In [0]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader): # distributed dataset
        
        model.send(data.location) # send the model to the right location
        
        #data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        model.get() # get the model back
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    print('finished training')  

### Instantiation,  hyperparams, model training

In [0]:
'''class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(3*28*28, 1024)
        self.fc2 = nn.Linear(1024, 102)

    def forward(self, x):
        #print(x.shape)
        x = x.view(-1, 3*28*28)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)'''

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 32
        self.test_batch_size = 1000
        self.epochs = 2
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()
print(torch.cuda.is_available())

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# model = Model()
model = models.densenet161(pretrained=True)
#model.to(device)
#model.classifier

optimizer = optim.SGD(model.parameters(), lr=args.lr) # momentum is not supported yet
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 4)



True


Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth
100%|██████████| 115730790/115730790 [00:04<00:00, 23889545.17it/s]


In [0]:
for param in model.parameters():
    param.requires_grad = True
    
fc_in = model.classifier.in_features

transferclassifier = nn.Sequential(
                        #nn.BatchNorm1d(fc_in),
                        nn.Linear(fc_in, 102)
                        )

model.classifier = transferclassifier
#model.fc = transferclassifier # resnet
model


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inpla

In [0]:
# create workers, 
hook = sy.TorchHook(torch)

ada = sy.VirtualWorker(hook, 'ada')
bob = sy.VirtualWorker(hook, 'bob')
cyd = sy.VirtualWorker(hook, 'cyd')

In [0]:
# Create the data loaders, federated PySyft loaders are returned
my_trainset, my_validset , _ = create_loaders('Densenet')
train_loader = dataset_federate(my_trainset, (ada,bob,cyd))
#valid_loader = dataset_federate(my_validset, (ada,bob,cyd))

returning datasets
dataset_federate


In [0]:
# for testing

print(train_loader.workers)
data, labels = next(iter(train_loader))
print(data)

print(f'objects of ada= {len(ada._objects)}, bob= {len(bob._objects)}, cyd= {len(cyd._objects)}')


['ada', 'bob', 'cyd']
(Wrapper)>[PointerTensor | me:21052157215 -> ada:13834107779]
objects of ada= 4, bob= 2, cyd= 2


### start the training

In [0]:
print(f'objects of ada= {len(ada._objects)}, bob= {len(bob._objects)}, cyd= {len(cyd._objects)}')
print(device)
epochs = 2
#model.to(device)

##### START THE TRAINING #### 
trainedmodel = train(args, model, device, train_loader, optimizer, epochs)

objects of ada= 4, bob= 2, cyd= 2
cuda


RuntimeError: ignored

### Clear the worker

In [0]:
#ada.clear_objects()
#bob.clear_objects()
#cyd.clear_objects()

<VirtualWorker id:cyd #objects:0>